In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load dataset
csv_path = '/mnt/data/my_dataset.csv'
df = pd.read_csv(csv_path, parse_dates=['Day'])
# Ensure columns are named as expected
print('Loaded', csv_path)
print('Shape:', df.shape)
print('\nColumn dtypes:')
print(df.dtypes)
print('\nMissing values per column:')
print(df.isnull().sum())


In [ ]:
# Ensure numeric columns
numeric_cols = ['Sessions','Duration','Transactions','Spent','Revenue']
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Basic cleaning: drop rows without UserID or Day
df = df.dropna(subset=['UserID','Day'])

# Quick head
print(df.head().to_string(index=False))


In [ ]:
# 1) Distinct users
distinct_users = df['UserID'].nunique()
print('Distinct users:', distinct_users)


In [ ]:
# 2) Totals and averages
total_sessions = df['Sessions'].sum()
total_transactions = df['Transactions'].sum()
total_revenue = df['Revenue'].sum()
total_spent = df['Spent'].sum()
avg_sessions_per_row = df['Sessions'].mean()
avg_duration = df['Duration'].mean()

print('Total sessions:', total_sessions)
print('Total transactions:', total_transactions)
print('Total revenue (USD):', total_revenue)
print('Total spent (USD):', total_spent)
print('Average sessions per row:', avg_sessions_per_row)
print('Average session duration (s):', avg_duration)


In [ ]:
# 3) Sessions / users / transactions by country
by_country = df.groupby('Country').agg(
    users=('UserID', 'nunique'),
    sessions=('Sessions','sum'),
    transactions=('Transactions','sum'),
    revenue=('Revenue','sum'),
    spent=('Spent','sum')
).reset_index()
print(by_country.to_string(index=False))


In [ ]:
# 4) OS breakdown
by_os = df.groupby('OS').agg(
    users=('UserID','nunique'),
    sessions=('Sessions','sum'),
    transactions=('Transactions','sum'),
    revenue=('Revenue','sum')
).reset_index()
print(by_os.to_string(index=False))


In [ ]:
# 5) Conversion metrics
# Define conversion as transactions / sessions (if sessions > 0)
conv_overall = total_transactions / total_sessions if total_sessions>0 else np.nan
conv_by_country = (df.groupby('Country').agg(transactions=('Transactions','sum'), sessions=('Sessions','sum'))
                   .assign(conversion=lambda x: x['transactions'] / x['sessions'].replace(0, np.nan)).reset_index())
print('Overall conversion (transactions / sessions):', conv_overall)
print('\nConversion by country:')
print(conv_by_country.to_string(index=False))


In [ ]:
# 6) Revenue per user (ARPU) and revenue per paying user
revenue_per_user = df.groupby('UserID')['Revenue'].sum().reset_index()
ARPU = revenue_per_user['Revenue'].mean()

paying_users = revenue_per_user[revenue_per_user['Revenue']>0]
ARPPU = paying_users['Revenue'].mean() if len(paying_users)>0 else 0

print('ARPU (mean revenue per user):', ARPU)
print('ARPPU (mean revenue among paying users):', ARPPU)
print('Number of paying users:', len(paying_users))


In [ ]:
# 7) Daily time series: total revenue and sessions per day
daily = df.groupby('Day').agg(
    sessions=('Sessions','sum'),
    transactions=('Transactions','sum'),
    revenue=('Revenue','sum')
).sort_index().reset_index()
print(daily.head().to_string(index=False))

# Plot daily revenue
plt.figure(figsize=(10,4))
plt.plot(daily['Day'], daily['revenue'])
plt.title('Daily total revenue')
plt.xlabel('Day')
plt.ylabel('Revenue (USD)')
plt.tight_layout()
plt.show()


In [ ]:
# 8) Bar chart: revenue by country
country_rev = by_country.sort_values('revenue', ascending=False)
plt.figure(figsize=(8,4))
plt.bar(country_rev['Country'], country_rev['revenue'])
plt.title('Revenue by Country')
plt.xlabel('Country')
plt.ylabel('Revenue (USD)')
plt.tight_layout()
plt.show()


In [ ]:
# 9) Bar chart: distinct users by country
country_users = by_country.sort_values('users', ascending=False)
plt.figure(figsize=(8,4))
plt.bar(country_users['Country'], country_users['users'])
plt.title('Distinct users by Country')
plt.xlabel('Country')
plt.ylabel('Users')
plt.tight_layout()
plt.show()


In [ ]:
# 10) OS share (pie-like bar) - show as bar
os_share = by_os.sort_values('users', ascending=False)
plt.figure(figsize=(6,4))
plt.bar(os_share['OS'], os_share['users'])
plt.title('Users by OS')
plt.xlabel('OS')
plt.ylabel('Users')
plt.tight_layout()
plt.show()


In [ ]:
# 11) Distribution of session durations (histogram)
plt.figure(figsize=(8,4))
plt.hist(df['Duration'].dropna(), bins=50)
plt.title('Session Duration Distribution (seconds)')
plt.xlabel('Duration (s)')
plt.ylabel('Count')
plt.tight_layout()
plt.show()


In [ ]:
# 12) Scatter: total spent vs revenue aggregated by UserID
user_agg = df.groupby('UserID').agg(spent=('Spent','sum'), revenue=('Revenue','sum'), sessions=('Sessions','sum')).reset_index()
plt.figure(figsize=(6,5))
plt.scatter(user_agg['spent'], user_agg['revenue'])
plt.title('User: Spent vs Revenue')
plt.xlabel('Total Spent (USD)')
plt.ylabel('Total Revenue (USD)')
plt.tight_layout()
plt.show()


In [ ]:
# 13) Top 10 users by revenue
top_users = user_agg.sort_values('revenue', ascending=False).head(10)
print(top_users.to_string(index=False))


In [ ]:
# 14) Save summary tables to CSVs (optional outputs)
out_dir = '/mnt/data/analysis_outputs'
os.makedirs(out_dir, exist_ok=True)
by_country.to_csv(os.path.join(out_dir, 'by_country.csv'), index=False)
by_os.to_csv(os.path.join(out_dir, 'by_os.csv'), index=False)
daily.to_csv(os.path.join(out_dir, 'daily.csv'), index=False)
user_agg.to_csv(os.path.join(out_dir, 'user_agg.csv'), index=False)
print('Saved summary CSVs to', out_dir)
